<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# LangGraph Router Pattern: Intent-Based Agent Routing
In this tutorial, we'll explore the Router pattern using LangGraph, a framework for building dynamic, stateful LLM applications. We’ll build a smart support assistant that can route customer queries to the appropriate specialized agent—Billing, Tech Support, or General Info—based on the user’s intent.

LangGraph enables this by letting us define a structured graph of logic, where a router node classifies the input, and conditional edges forward it to the correct sub-agent. Each agent uses local context (e.g., invoice history, troubleshooting tips) to respond intelligently.

We also trace this application using Phoenix, which gives us complete visibility into routing decisions, tool usage, and model interactions. This is helpful for debugging routing accuracy and understanding how the graph executes end-to-end.

In [1]:
!pip install langgraph langchain langchain_community "arize-phoenix==9.0.1" arize-phoenix-otel openinference-instrumentation-langchain

In [2]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.1 MB/s eta 0:00:00


In [3]:
from langgraph.graph import StateGraph, START, END
import os, getpass

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


# Configure Phoenix Tracing

Make sure you go to https://app.phoenix.arize.com/ and generate an API key. This will allow you to trace your Langgraph application with Phoenix.

In [25]:
PHOENIX_API_KEY = getpass.getpass("Phoenix API Key:")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

Phoenix API Key:··········


In [6]:
from phoenix.otel import register

tracer_provider = register(
  project_name="Router",
  auto_instrument=True
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Router
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [7]:
from typing_extensions import TypedDict, Literal
from IPython.display import Image, display

from langchain_openai import ChatOpenAI

# LLM

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

In [9]:
from typing import Literal, TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import SystemMessage, HumanMessage


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Router
We define a Pydantic schema to structure the router’s output. This schema ensures the LLM returns one of three valid routing targets: "billing", "tech_support", or "general_info".
We then wrap our LLM with with_structured_output, allowing LangGraph to enforce this structured response during routing.

In [10]:
class Route(BaseModel):
    step: Literal["billing", "tech_support", "general_info"] = Field(description="Classify the support request")

router = llm.with_structured_output(Route)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1630: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


# Defining Graph State
This state schema captures the lifecycle of a routed request. It stores the original user input (input), the classification decision made by the router (decision), and the final response generated by the appropriate support handler (output). Each node in the graph will read from and write to this shared state.

In [11]:
class State(TypedDict):
    input: str
    decision: str
    output: str

# Support Agent Nodes: Specialized Response Handlers
This section defines three specialized LLM-powered agents, each responsible for handling a different category of user queries:

**Billing Agent**: Uses user billing history and a billing policy context to respond to invoice or refund-related questions.

**Tech Support Agent**: Answers common troubleshooting queries using a predefined support knowledge base.

**General Info Agent**: Responds to account, subscription, and policy-related questions using general FAQs.

Each agent reads the input field from the graph's state and returns a generated output tailored to its domain. These agents form the execution endpoints of the router graph.

In [12]:
# Billing Agent
billing_history_data = [
    {"invoice_id": "INV001", "date": "2024-11-01", "amount": "$29.99"},
    {"invoice_id": "INV002", "date": "2024-12-01", "amount": "$29.99"},
    {"invoice_id": "INV003", "date": "2025-01-01", "amount": "$39.99"},
]

billing_general_context = (
    "Billing inquiries may include refunds, invoices, plan upgrades, or charges. "
    "Our system charges users monthly based on their plan. Refunds are processed within 5–7 business days."
)

def billing_agent(state: State):
    user_input = state["input"]
    invoice_summary = "\n".join(
        f"• Invoice {item['invoice_id']} on {item['date']} for {item['amount']}"
        for item in billing_history_data
    )
    prompt = (
        f"You are a helpful billing assistant.\n"
        f"Here is the user's billing history:\n{invoice_summary}\n\n"
        f"General billing context:\n{billing_general_context}\n\n"
        f"User query:\n{user_input}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}


# Tech Support Agent
tech_support_kb = (
    "Common issues include login errors, app crashes, and network connectivity. "
    "To fix login errors, check your email and reset your password. "
    "For crashes, try reinstalling the app. If your connection is unstable, restart your router."
)

def tech_support_agent(state: State):
    prompt = (
        "You are a tech support assistant. Use the knowledge base below to answer the user's question.\n\n"
        f"Knowledge Base:\n{tech_support_kb}\n\n"
        f"User query:\n{state['input']}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}


# General Info Agent
general_info_kb = (
    "We offer 3 subscription plans: Basic, Pro, and Enterprise. "
    "Support is available 24/7. You can cancel your subscription any time from the account settings page."
)

def general_info_agent(state: State):
    prompt = (
        "You are a general info assistant. Use the knowledge base below to answer the user's question.\n\n"
        f"Knowledge Base:\n{general_info_kb}\n\n"
        f"User query:\n{state['input']}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}



# Intent Classification & Routing Logic
This section introduces the decision-making logic that powers the routing mechanism:

**classify_intent Node**: Uses an LLM wrapped with a structured output schema to classify the user's query into one of three support categories — billing, tech_support, or general_info. The result is stored in the state's decision key.

**route_to_agent Function**: A conditional router that examines the classification result and sends the request to the corresponding specialized agent node. This allows LangGraph to dynamically direct queries to the most relevant response module.

In [13]:
def classify_intent(state: State):
    decision = router.invoke(
        [
            SystemMessage(content="Classify this support request as billing, tech_support, or general_info."),
            HumanMessage(content=state["input"]),
        ]
    )
    return {"decision": decision.step}

def route_to_agent(state: State):
    if state["decision"] == "billing":
        return "billing_agent"
    elif state["decision"] == "tech_support":
        return "tech_support_agent"
    elif state["decision"] == "general_info":
        return "general_info_agent"


# Building the Routing Graph
Here, we construct the complete LangGraph workflow by registering the nodes and defining their connections:

**Node Registration**: All agents (billing_agent, tech_support_agent, general_info_agent) and the classify_intent node are added to the graph.

**Conditional Routing**: After the graph starts at classify_intent, it uses the output of route_to_agent to forward the query to the appropriate agent node based on the classified intent.

**Terminal Edges**: Each agent node directly leads to END, finalizing the workflow after responding to the query.

This design reflects a typical customer support router architecture, enabling modular, extensible handling of diverse query types.

In [14]:
builder = StateGraph(State)

builder.add_node("classify_intent", classify_intent)
builder.add_node("billing_agent", billing_agent)
builder.add_node("tech_support_agent", tech_support_agent)
builder.add_node("general_info_agent", general_info_agent)

builder.add_edge(START, "classify_intent")

builder.add_conditional_edges(
    "classify_intent",
    route_to_agent,
    {
        "billing_agent": "billing_agent",
        "tech_support_agent": "tech_support_agent",
        "general_info_agent": "general_info_agent",
    },
)

builder.add_edge("billing_agent", END)
builder.add_edge("tech_support_agent", END)
builder.add_edge("general_info_agent", END)

workflow = builder.compile()

# Let's run some queries:

In [15]:
queries = [
    "Why was I charged $39.99 this month?",                    # billing
    "The app keeps crashing when I open it.",                  # tech support
    "Can I cancel my subscription anytime?",                   # general info
    "Can you show me all my past invoices?",                   # billing
    "How do I fix login issues with my account?"               # tech support
]

# Run each query through the router
for i, query in enumerate(queries, start=1):
    print(f"\n--- Query {i} ---")
    state = workflow.invoke({"input": query})
    print(f"Input: {query}")
    print(f"Response:\n{state['output']}")


--- Query 1 ---
Input: Why was I charged $39.99 this month?
Response:
You were charged $39.99 this month for Invoice INV003, which was issued on January 1, 2025. This charge reflects an increase from your previous monthly invoices of $29.99. If you have any questions about the reason for the increase or if you believe there was an error, please let me know!

--- Query 2 ---
Input: The app keeps crashing when I open it.
Response:
If the app keeps crashing when you open it, try reinstalling the app. This often resolves issues related to crashes. If the problem persists after reinstalling, please let me know for further assistance.

--- Query 3 ---
Input: Can I cancel my subscription anytime?
Response:
Yes, you can cancel your subscription anytime from the account settings page.

--- Query 4 ---
Input: Can you show me all my past invoices?
Response:
Sure! Here are your past invoices:

1. **Invoice INV001**
   - Date: 2024-11-01
   - Amount: $29.99

2. **Invoice INV002**
   - Date: 2024-1

# Make sure to view your traces in Phoenix!

# Evals

In this section we will add a simple eval that evaluates whether the router chose the correct subagent. We will use the tool calling eval template from Phoenix, as routing to sub agents mimics the process of picking a tool to call.

In [16]:
from phoenix.evals import (
    TOOL_CALLING_PROMPT_RAILS_MAP,
    TOOL_CALLING_PROMPT_TEMPLATE,
    OpenAIModel,
    llm_classify,
)

## Get all root spans

In [119]:
df = px.Client().get_spans_dataframe("name == 'LangGraph'", project_name='Router')
df

/usr/local/lib/python3.11/dist-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.27.0) and client (9.0.1) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,attributes.input.value,attributes.output.value,attributes.openinference.span.kind,attributes.output.mime_type
context.span_id,,,,,,,,,,,,,,
2494c558d7ee55a8,LangGraph,CHAIN,None,2025-05-04 20:04:10.552909+00:00,2025-05-04 20:04:14.802115+00:00,OK,,[],2494c558d7ee55a8,2c96b5be480ea2e1c7ec38034214541d,Why was I charged $39.99 this month?,"{""input"": ""Why was I charged $39.99 this month...",CHAIN,application/json
3089f28d26564377,LangGraph,CHAIN,None,2025-05-04 20:04:14.967851+00:00,2025-05-04 20:04:18.399871+00:00,OK,,[],3089f28d26564377,2b90804ebbbf9cdeff3a19cfe8b05c23,The app keeps crashing when I open it.,"{""input"": ""The app keeps crashing when I open ...",CHAIN,application/json
41a6f7ebdd0121a7,LangGraph,CHAIN,None,2025-05-04 20:04:18.563250+00:00,2025-05-04 20:04:20.926127+00:00,OK,,[],41a6f7ebdd0121a7,7ea1240db90424fabc4ef1baf1aeca8b,Can I cancel my subscription anytime?,"{""input"": ""Can I cancel my subscription anytim...",CHAIN,application/json
342107df1aa5cc81,LangGraph,CHAIN,None,2025-05-04 20:04:21.092434+00:00,2025-05-04 20:04:25.506294+00:00,OK,,[],342107df1aa5cc81,abeb4c809649e9f7272f954440f6e47a,Can you show me all my past invoices?,"{""input"": ""Can you show me all my past invoice...",CHAIN,application/json
43ae9b9b83eea783,LangGraph,CHAIN,None,2025-05-04 20:04:25.669565+00:00,2025-05-04 20:04:28.843946+00:00,OK,,[],43ae9b9b83eea783,c70e73d8ec984d8c10fb24d358f57408,How do I fix login issues with my account?,"{""input"": ""How do I fix login issues with my a...",CHAIN,application/json
8b1e1743c30a1845,LangGraph,CHAIN,None,2025-05-14 23:58:07.512731+00:00,2025-05-14 23:58:11.064226+00:00,OK,,[],8b1e1743c30a1845,d21d266247d4cdc232906119ae9eea6b,Why was I charged $39.99 this month?,"{""input"": ""Why was I charged $39.99 this month...",CHAIN,application/json
d9e5859a98b371b7,LangGraph,CHAIN,None,2025-05-14 23:58:11.112152+00:00,2025-05-14 23:58:13.219407+00:00,OK,,[],d9e5859a98b371b7,d4a9da42d7a82a2540c3fa0062e882b5,The app keeps crashing when I open it.,"{""input"": ""The app keeps crashing when I open ...",CHAIN,application/json
614858aa30fe8a38,LangGraph,CHAIN,None,2025-05-14 23:58:13.263072+00:00,2025-05-14 23:58:14.729684+00:00,OK,,[],614858aa30fe8a38,2dcb24cbb978c7a2215941add6076721,Can I cancel my subscription anytime?,"{""input"": ""Can I cancel my subscription anytim...",CHAIN,application/json
a75616c145ef3942,LangGraph,CHAIN,None,2025-05-14 23:58:14.776334+00:00,2025-05-14 23:58:17.866425+00:00,OK,,[],a75616c145ef3942,2df78f40dab4df37ad54cf44a5679afd,Can you show me all my past invoices?,"{""input"": ""Can you show me all my past invoice...",CHAIN,application/json


In [120]:
df.rename(columns={'attributes.output.value': 'tool_call'}, inplace=True)
df.rename(columns = {"attributes.input.value": "question"}, inplace=True)

In [121]:
df_clean = df[['context.span_id', "question", "tool_call"]]
df_clean

,context.span_id,question,tool_call
context.span_id,,,
2494c558d7ee55a8,2494c558d7ee55a8,Why was I charged $39.99 this month?,"{""input"": ""Why was I charged $39.99 this month..."
3089f28d26564377,3089f28d26564377,The app keeps crashing when I open it.,"{""input"": ""The app keeps crashing when I open ..."
41a6f7ebdd0121a7,41a6f7ebdd0121a7,Can I cancel my subscription anytime?,"{""input"": ""Can I cancel my subscription anytim..."
342107df1aa5cc81,342107df1aa5cc81,Can you show me all my past invoices?,"{""input"": ""Can you show me all my past invoice..."
43ae9b9b83eea783,43ae9b9b83eea783,How do I fix login issues with my account?,"{""input"": ""How do I fix login issues with my a..."
8b1e1743c30a1845,8b1e1743c30a1845,Why was I charged $39.99 this month?,"{""input"": ""Why was I charged $39.99 this month..."
d9e5859a98b371b7,d9e5859a98b371b7,The app keeps crashing when I open it.,"{""input"": ""The app keeps crashing when I open ..."
614858aa30fe8a38,614858aa30fe8a38,Can I cancel my subscription anytime?,"{""input"": ""Can I cancel my subscription anytim..."
a75616c145ef3942,a75616c145ef3942,Can you show me all my past invoices?,"{""input"": ""Can you show me all my past invoice..."


## Add our subagent definitions as tools

Each column must contain information regarding the tools that could have been chosen from. Since our "tools" in this case are our subagents, we pass in the definitions (descriptions) of our subagents.

In [122]:
subagent_definitions = """
 billing: Uses user billing history and a billing policy context to respond to invoice or refund-related questions.
tech_support: Answers common troubleshooting queries using a predefined support knowledge base.
general_info: Responds to account, subscription, and policy-related questions using general FAQs.
"""
df_clean["tool_definitions"] = subagent_definitions

<ipython-input-122-da26864dd2f9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["tool_definitions"] = tool_definitions


In [123]:
df_clean

,context.span_id,question,tool_call,tool_definitions
context.span_id,,,,
2494c558d7ee55a8,2494c558d7ee55a8,Why was I charged $39.99 this month?,"{""input"": ""Why was I charged $39.99 this month...",\n billing: Uses user billing history and a b...
3089f28d26564377,3089f28d26564377,The app keeps crashing when I open it.,"{""input"": ""The app keeps crashing when I open ...",\n billing: Uses user billing history and a b...
41a6f7ebdd0121a7,41a6f7ebdd0121a7,Can I cancel my subscription anytime?,"{""input"": ""Can I cancel my subscription anytim...",\n billing: Uses user billing history and a b...
342107df1aa5cc81,342107df1aa5cc81,Can you show me all my past invoices?,"{""input"": ""Can you show me all my past invoice...",\n billing: Uses user billing history and a b...
43ae9b9b83eea783,43ae9b9b83eea783,How do I fix login issues with my account?,"{""input"": ""How do I fix login issues with my a...",\n billing: Uses user billing history and a b...
8b1e1743c30a1845,8b1e1743c30a1845,Why was I charged $39.99 this month?,"{""input"": ""Why was I charged $39.99 this month...",\n billing: Uses user billing history and a b...
d9e5859a98b371b7,d9e5859a98b371b7,The app keeps crashing when I open it.,"{""input"": ""The app keeps crashing when I open ...",\n billing: Uses user billing history and a b...
614858aa30fe8a38,614858aa30fe8a38,Can I cancel my subscription anytime?,"{""input"": ""Can I cancel my subscription anytim...",\n billing: Uses user billing history and a b...
a75616c145ef3942,a75616c145ef3942,Can you show me all my past invoices?,"{""input"": ""Can you show me all my past invoice...",\n billing: Uses user billing history and a b...


# Extracting the subagent that was chosen.

In [124]:
import re
def extract_decision(x):
    """Extracts the decision value from a string using regex.

    Handles cases where the regex search returns None.
    """
    if x is None:
        return None
    match = re.search(r'"decision"\s*:\s*"([^"]+)"', x)
    if match:
        return match.group(1)
    else:
        return None  # Or any default value you prefer

df_clean["tool_call"] = df_clean["tool_call"].apply(extract_decision)
df_clean

<ipython-input-124-9e044e916bca>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["tool_call"] = df_clean["tool_call"].apply(extract_decision)


,context.span_id,question,tool_call,tool_definitions
context.span_id,,,,
2494c558d7ee55a8,2494c558d7ee55a8,Why was I charged $39.99 this month?,billing,\n billing: Uses user billing history and a b...
3089f28d26564377,3089f28d26564377,The app keeps crashing when I open it.,tech_support,\n billing: Uses user billing history and a b...
41a6f7ebdd0121a7,41a6f7ebdd0121a7,Can I cancel my subscription anytime?,general_info,\n billing: Uses user billing history and a b...
342107df1aa5cc81,342107df1aa5cc81,Can you show me all my past invoices?,billing,\n billing: Uses user billing history and a b...
43ae9b9b83eea783,43ae9b9b83eea783,How do I fix login issues with my account?,tech_support,\n billing: Uses user billing history and a b...
8b1e1743c30a1845,8b1e1743c30a1845,Why was I charged $39.99 this month?,billing,\n billing: Uses user billing history and a b...
d9e5859a98b371b7,d9e5859a98b371b7,The app keeps crashing when I open it.,tech_support,\n billing: Uses user billing history and a b...
614858aa30fe8a38,614858aa30fe8a38,Can I cancel my subscription anytime?,general_info,\n billing: Uses user billing history and a b...
a75616c145ef3942,a75616c145ef3942,Can you show me all my past invoices?,billing,\n billing: Uses user billing history and a b...


In [125]:
df_clean.to_csv('router_evals.csv', index=False)

# Running Evals

In [126]:
eval_model = OpenAIModel(model="gpt-4o")

In [127]:
rails = list(TOOL_CALLING_PROMPT_RAILS_MAP.values())

response_classifications = llm_classify(
    dataframe=df_clean,
    template=TOOL_CALLING_PROMPT_TEMPLATE,
    model=eval_model,
    rails=rails,
    provide_explanation=True,
)
response_classifications["score"] = (response_classifications["label"] == "correct").astype(int)

<ipython-input-127-bb4cab0229e0>:3: DeprecationWarning: `dataframe` argument is deprecated; use `data` instead
  response_classifications = llm_classify(


llm_classify |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

In [128]:
response_classifications

,label,explanation,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,,
2494c558d7ee55a8,correct,The question asks why the user was charged $39...,[],COMPLETED,2.442497,1
3089f28d26564377,correct,"The question is about an app crashing issue, w...",[],COMPLETED,1.953106,1
41a6f7ebdd0121a7,correct,"The question ""Can I cancel my subscription any...",[],COMPLETED,1.527270,1
342107df1aa5cc81,correct,"The question asks for past invoices, which is ...",[],COMPLETED,2.020009,1
43ae9b9b83eea783,correct,The question is about fixing login issues with...,[],COMPLETED,1.714982,1
8b1e1743c30a1845,correct,The question asks why the user was charged $39...,[],COMPLETED,2.032856,1
d9e5859a98b371b7,correct,"The question is about an app crashing, which i...",[],COMPLETED,1.592437,1
614858aa30fe8a38,correct,The question asks about the ability to cancel ...,[],COMPLETED,2.038828,1
a75616c145ef3942,correct,"The question asks for past invoices, which is ...",[],COMPLETED,1.503455,1


# Exporting Evals to Phoenix!

In [129]:
response_classifications.drop(columns=["exceptions", "execution_status", "execution_seconds"], inplace=True)

In [130]:
response_classifications.to_parquet("router_evals.parquet")

In [131]:
from phoenix.trace import SpanEvaluations


df = response_classifications.copy()
df.index.name = "span_id"
px_client.log_evaluations(
    SpanEvaluations(
        eval_name = "Routing Eval",
        dataframe = df,
    )
)



/usr/local/lib/python3.11/dist-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.27.0) and client (9.0.1) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(
